In [1]:
import numpy as np 

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit import Aer
from qiskit.primitives import Sampler, BackendSampler
from qiskit.circuit.library import TwoLocal

from src.Quantum_Optimizer_FGS_modified import Quantum_Optimizer_FGS

In [2]:
target_mdl = QuadraticProgram()
target_mdl.read_from_lp_file('../lp_files/Random_seq13.lp')

N = target_mdl.get_num_vars()
print(N)

#print(target_mdl.prettyprint())

64


In [3]:
cplex_result = CplexOptimizer().solve(target_mdl)
cplex_optval = cplex_result.fval
cplex_xsol = cplex_result.x.astype(int)

print(
        '-'*110,
        '[CPLEX Result]',
        f'   optimal value: {cplex_optval:.3f}',
        f'   optimal solution (binary): {cplex_xsol}',
        f'   status: {cplex_result.status.name}',
        '-'*110,
        sep='\n'
    )

--------------------------------------------------------------------------------------------------------------
[CPLEX Result]
   optimal value: -118.200
   optimal solution (binary): [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0
 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
   status: SUCCESS
--------------------------------------------------------------------------------------------------------------


In [15]:
qubo = QuadraticProgramToQubo().convert(target_mdl)

import matplotlib.pyplot as plt

dict_J = qubo.objective.linear.to_dict().values()

plt.plot(dict_J, 'o')
#print(qubo.prettyprint())

In [16]:
arr = qubo.objective.quadratic.to_array()
indx_50 = []
indx_20 = []

for idx, x in np.ndenumerate(arr):
    if x == 50.0:
        for elem in idx:
            indx_50.append(elem)
    else:
        for elem in idx:
            indx_20.append(elem)

In [17]:
np.allclose(arr, np.triu(arr)) # check if lower triangular
#np.allclose(mat, np.triu(mat))

True

In [18]:
num_qubits = N
print(f'number of system qubits: {num_qubits}')

number of system qubits: 118


In [19]:
reps = 2

ansatz = TwoLocal(
    num_qubits = num_qubits,
    rotation_blocks = 'ry', 
    entanglement_blocks = 'cz',
    entanglement = 'pairwise', 
    reps = reps,
    insert_barriers = True
    )

n_params = ansatz.num_parameters
print(f'number of params: {n_params}')

number of params: 354


In [20]:
shots = 2**10
backend = Aer.get_backend('aer_simulator_matrix_product_state')

sampler = BackendSampler(backend=backend)
sampler.set_options(shots=shots)

print(f'shots = {shots}')

shots = 1024


In [21]:
J = target_mdl.objective.quadratic.coefficients.copy()
h = target_mdl.objective.linear
    
for k in range(N):
    J[k,k] += h[k]

J = J.toarray()
J_positive =  J[J>0]   


def objective(x, alpha):
    
    return x @ (J_positive * alpha) @ x

In [22]:
print(f'reference value: {cplex_optval:.3f}')
print('-'*80)

QuantumOptimizer = Quantum_Optimizer_FGS()
QuantumOptimizer.set_optimizer(num_qubits, ansatz, sampler)

theta_initial = np.hstack([np.full(num_qubits, np.pi/2), np.zeros(num_qubits*ansatz.reps)])
max_energy =  np.sum([abs(elem) for elem in qubo.objective.quadratic.to_dict().values() if elem >= 0.0])
print(max_energy)

#filter_func = {"exp": np.exp(x), "power"=x**(0.7), "log"=np.ln(x)}
result = QuantumOptimizer.run(
    objective = objective, 
    max_energy = max_energy,
    op_trans = lambda x: -x,
    qc_params_init = theta_initial,
    max_epoch = 10, 
    random_update = True, 
    intermediate_result_display = True,
    refval = cplex_optval
)

reference value: -129.500
--------------------------------------------------------------------------------
262600.0


KeyboardInterrupt: 

In [ ]:
x = QuantumOptimizer.best_x
objval = qubo.objective.evaluate(x)

print(
        '-'*110,
        '[Quantum Result]',
        f'   objective:   {objval:.3f}',
        f'   solution:    {x}',
        f'   is feasible: {target_mdl.is_feasible(x)}',
        '-'*110,
        sep='\n'
    )